# Clustering

Clustering merupakan sebuah teknik yang digunakan untuk menemukan sub-kelompok atau kluster. Prinsip dari clustering adalah data-data yang termasuk dalam kluster yang sama “diharapkan” memiliki kesamaan satu dengan yang lainnya, sedangkan data-data yang ada dalam kluster yang berbeda cukup berbeda dari satu kluster dengan cluster yang lainnya. Meskipun tanpa label, clustering memanfaatkan “karakter/pola” pada data bahwa data yang “mirip” akan berdekatan, seperti pada gambar di samping.



Bertujuan untuk membuat segmentasi customer yang dimana merupakan proses membagi pelanggan berdasarkan karakteristik umum, seperti demografi atau perilaku, sehingga perusahaan dapat melakukan pemasaran secara lebih efektif.

Beberapa keuntungan dari segmentasi customer seperti:

1.membuat kampanye marketing yang terkustomisasi

2.menentukan fitur apa yang bisa dirilis untuk customer tertentu

3.membuat prioritas dalam pengembangan produk

4.menentukan harga produk sesuai dengan segmen customer dan lainnya

In [2]:
from pprint import pprint
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
df_transactions = pd.read_csv ("transactions.csv")
df_articles     = pd.read_csv ("articles.csv")
df_customers    = pd.read_csv ("customers.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'transactions.csv'

In [ ]:
print("num of articles:", df_articles.shape[0])
print("num of customers:", df_customers.shape[0])
print("num of transactions:", df_transactions.shape[0])

### Data Articles

In [ ]:
with pd.option_context("display.max_columns", None):
    display(df_articles.head())

In [ ]:
# Pilih indeks pertama pada df_article dan dari sini kita dapat mengetahui kira-kira apa isi data dari tiap kolom
df_articles.loc[0]

Terdapat 25 fitur yang merepresentasikan informasi tentang artikel tersebut, yaitu nama produk, warna produk, deskripsi produk, dan sebagainya.
Sayangnya, tidak ada data dictionary yang tersedia, sehingga kita sedikit kesulitan mengartikan masing-masing fitur.

In [ ]:
df_articles.info()

#Hampir semua fitur tidak memiliki missing values. 
#Hanya fitur detail_desc yang memiliki beberapa missing values (105126 dari 105542)

### Data Customers

Berisi informasi-informasi tentang customer

In [ ]:
df_customers

In [ ]:
df_customers.info()

Dari informasi di atas, terlihat bahwa hampir semua fitur yang ada pada df_customers memiliki missing values. 

Fitur yang paling banyak memiliki missing values adalah Active, sedangkan fitur yang tidak memiliki missing values adalah customer_id dan postal_code. 

Sama seperti data artikel, dikarenakan tidak ada data dictionary yang tersedia, sehingga kita akan sedikit kesulitan mengartikan masing-masing fitur dan untuk mengetahui representasi masing-masing fitur, kita perlu melihat nilai-nilai yang ada di masing-masing fitur tersebut

### Data Transactions

In [ ]:
with pd.option_context("display.max_columns", None):
    display(df_transactions.head())

In [ ]:
df_transactions.info()

Pada data transaksi, terdapat 5 fitur yang merepresentasikan sebuah transaksi, seperti tanggal transaksi, nomor customer, dan nomor artikel, harga artikel, dan channel penjualan.

Karena jumlah baris yang sangat banyak (~31 juta baris), fungsi info tidak bisa menampilkan jumlah non-missing values sehingga lebih baik menggunakan kode isna().sum() untuk setiap fitur pada data transaksi, tidak terdapat missing values sama sekali

## Exploratory Data Analysis

1.Terdapat 3 dataset yang saling melengkapi satu sama lain, kita akan melakukan eksplorasi untuk ketiga dataset tersebut.

2.Khusus untuk eksplorasi pada data transaksi, kita akan gabungkan terlebih dahulu data customer dan artikel dengan data transaksi, sehingga setiap baris pada data transaksi akan terdiri dari gabungan fitur-fitur dari data customer, artikel, dan transaksi

Langkah-langkah yang akan kita lakukan untuk EDA adalah sebagai berikut:

-Eksplor data artikel, seperti jumlah (unik) artikel yang tersedia, jenis warna yang tersedia pada suatu produk, jenis garmen, dan lainnya.

-Eksplor data customer, seperti rentang umur customer, jumlah customer yang mendapatkan newsletter, status keanggotaan, dan lainnya.

-Eksplor data transaksi yang mencakup jumlah pembelian harian / bulanan / tahunan, produk terpopuler, CLV, dan lainnya.

### EDA Data Articles

In [ ]:
 df_articles.dtypes

Fitur article_id, product_code, product_type_no, dan sejenisnya bertipe data integer. Fitur tersebut seharusnya bertipe data kategorik karena tidak mencerminkan deret angka sehingga ubah tipe data fitur tersebut menjadi object.

In [ ]:
# Instruksi looping untuk mengubah tipe data 
# Pada kolom yang berakhiran dengan 'id','no', dan 'code' menggunakan function astype()

for feature in df_articles.columns:
    if (
        feature.endswith("id")
        or feature.endswith("no")
        or feature.endswith("code")
    ):
        df_articles[feature] = df_articles[feature].astype("object")

In [ ]:
df_articles.info()

In [ ]:
# Melihat unique values

for feature in df_articles.columns:
    print(
        f"- num of unique values in `{feature}`:",
        df_articles[feature].nunique()
    )

1. Terdapat perbedaan jumlah nilai unik antara beberapa pasangan kode dan nama, seperti product_code dengan prod_name, product_type_no dengan product_type_name, department_no dengan department_name, dan lainnya. Sehingga kita mungkin perlu melihat two-way table antara kedua fitur tersebut
2. karena index_group_name adalah fitur dengan jumlah nilai unik paling sedikit, yang mungkin berarti ada 5 grup besar yang membagi produk H&M

In [ ]:
def plot_count(data, x=None, y=None, palette=None, figsize=(12, 6), **kwargs):
    column_order = data[x or y].value_counts().index

    plt.figure(figsize=figsize)
    sns.countplot(
        x=x, y=y, data=data, order=column_order,
        palette=palette, **kwargs
    )
    plt.grid(False)


MAIN_COLOR = ["#29B5BF"]    # warna utama untuk visualisasi
OTHER_COLOR = ["#777777"]    # warna lainnya untuk visualisasi

In [ ]:
plot_count(x="index_group_name", data=df_articles)

 Insight :
 
 1. Ladieswear atau pakaian untuk perempuan menjadi kelompok grup yang paling banyak disediakan oleh H&M
 2. sebaliknya, ketersediaan pakaian olahraga atau 'sport' oleh H&M paling sedikit dibandingkan dengan yang kelompok pakaian lainnya
 3. Selanjutnya, kita lihat dari nama fitur yang mirip, yaitu index_group_name dengan index_name, sekarang kita akan coba bandingkan.

In [ ]:
plot_count(x="index_group_name", hue="index_name",data=df_articles, figsize=(15, 6))
plt.legend(loc="upper right")
plt.show()

Insight :
1. hanya kelompok Ladieswear dan Baby/Children yang memiliki sub-kelompok indeks
2. sub-kelompok dari Ladieswear terdiri dari:
    - Ladieswear
    - Lingeries/Tights
    - Ladies/Accessories
3. sub-kelompok dari Baby/Children terdiri dari:
    - Baby Sizes 50-98
    - Children Sizes 92-140
    - Children Sizes 134-170
    -Children Accessories, Swimwear

Kemudian, saat pelanggan memilih sebuah produk fashion, salah satu faktor yang menjadi pertimbangan adalah warna. Mari kita coba lihat warna apa saja yang disediakan oleh H&M.

In [ ]:
plot_count(y="perceived_colour_master_name", data=df_articles, figsize=(12, 8))
plt.xticks(rotation=90)
plt.show()

Insight :

1. Terlihat dari visualisasi di atas dapat diketahui 5 warna yang teratas adalah:
    - Black
    - Blue
    - White
    - Pink
    - Grey
2. Terdapat warna unknown dan undefined, maka warna ini perlu kita tinjau lebih jauh lagi tentang produk yang memiliki warna tersebut, apakah produk tsb sebuah pakaian ataukah aksesoris
3. Perihal warna, kita mungkin sadar bahwa pakaian pria dan pakaian wanita mungkin memiliki jenis warna yang berbeda jenisnya. Sehingga, menarik untuk kita lihat bagaimana H&M memilih warna-warna untuk kelompok produk Ladieswear dan Menswear.

In [ ]:
df_ladies_mens = df_articles[df_articles.index_group_name.isin(["Ladieswear", "Menswear"])
]
plot_count(
    hue="perceived_colour_master_name",
    x="index_group_name",
    data=df_ladies_mens,
    palette="Paired",
    figsize=(15, 8),
    hue_order=df_articles.perceived_colour_master_name.value_counts().index
)

Insight :
1. Untuk Ladieswear dan Menswear warna hitam menjadi warna yang populer 
2. 5 warna teratas untuk kelompok pakaian Ladieswear adalah:
    - Black
    - White
    - Blue
    - Beige
    - Pink
3. 5 warna teratas untuk kelompok pakaian Menswear adalah:
    - Black
    - Blue
    - Grey
    - White
    - Red
4. Kedua kelompok pakaian tersebut masih memiliki warna unknown dan undefined sehingga selanjutnya, mari kita lihat bagaimana warna unknown dan undefined ini bisa ada di 2 kelompok tersebut.


In [ ]:
df_ladies_mens.loc[df_ladies_mens.perceived_colour_master_name.isin(["unknown", "undefined"]),
    ["product_type_name", "product_group_name", "graphical_appearance_name",
     "perceived_colour_master_name", "index_name", "section_name"]].sample(10)

In [ ]:
plot_count(
    y="product_type_name",
    data=df_ladies_mens[df_ladies_mens.perceived_colour_master_name.isin(["unknown", "undefined"])],
    figsize=(12, 8)
)

### EDA Data Customers

Data Customers terdapat banyak nilai missing values , sehingga kita akan coba eksplor missing values tersebut namun karena alasan privasi, kita hanya bisa mengeksplor fitur-fitur seperti FN, Active, club_member_status, fashion_news_frequency, dan age.

### Missing Values

In [ ]:
df_customers.isnull().sum()

In [ ]:
df_customers[df_customers.isna().all(axis=1)]

Insights:

Dari data di atas, terlihat bahwa tidak ada baris yang semua nilai fiturnya kosong.

In [ ]:
#Pilih fitur yang memiliki missing values & tampilkan nilai missing values serta hitung jumlahnya dengan unique()

for feature in df_customers.columns:
    if df_customers[feature].isna().any():
        print(f"num of unique values in {feature}: ",
            df_customers[feature].unique())

Insight :
1. Khusus pada fitur FN dan Active, terlihat bahwa hanya ada 1 untuk nilai non-missing value.
______________________________________________________________________________________________

2. FN merepresentasikan apakah customer mendapatkan fashion newsletter atau tidak
3. Active merepresentasikan apakah customer aktif untuk berkomunikasi, kita bisa anggap nilai nan sama dengan 0.
4. Kita akan isi nilai nan pada kedua fitur tersebut dengan nilai 0.
______________________________________________________________________________________________

5. Selanjutnya, kita akan coba lihat missing values yang ada pada fitur club_member_status dan fashion_news_frequency. 
6. Khusus untuk fashion_news_frequency, terdapat tiga jenis nilai yang mungkin sama, yaitu NONE, None, dan nan. Untuk penyederhanaan, kita akan anggap ketiga nilai tersebut sama, yaitu nan (string). Kemudian, nilai nan tersebut akan diisi berdasarkan hubungan dengan 2 fitur yang lain, club_member_status dan FN.


In [ ]:
# Pilih kedua kolom dan isi missing value dengan 0
df_customers[["FN", "Active"]] = df_customers[["FN", "Active"]].fillna(0)

In [ ]:
# Cek missing values
df_customers[["FN", "Active"]].isna().sum()

In [ ]:
#Mengubah NONE, None menjadi nan

df_customers["fashion_news_frequency"] = (
    df_customers["fashion_news_frequency"].replace({"NONE": np.nan, "None": np.nan})
)

In [ ]:
customer_news_nan = df_customers.loc[df_customers["fashion_news_frequency"].isna(),
                                     ["FN", "club_member_status", "fashion_news_frequency"]].fillna("nan")

fig, axis = plt.subplots(1, 2, figsize=(15, 6))
for ax, x in zip(axis, ["FN", "club_member_status"]):
    sns.countplot(x=x, data=customer_news_nan, ax=ax)

In [ ]:
customer_news_nan.value_counts()

Insights:

* Dari visualisasi dan informasi di atas, bisa disimpulkan bahwa:
    - Semua customer dengan nilai nan pada fashion_news_frequency dan nilai FN=0, akan diisi dengan Never, dengan asumsi customer yang tidak mengikuti newsletter tidak akan pernah mendapatkan berita fashion dari H&M.
    - Semua customer dengan dengan nilai nan pada fashion_news_frequency, club_member_status yang bernilai nan atau ACTIVE, dan nilai FN=1 akan diisi dengan nilai Other.
    - Semua customer dengan club_member_status=PRE_CREATE dengan kondisi nilai FN=1diasumsikan bahwa customer tersebut masih dalam proses pembuatan akun / member, sehingga perlu jenis nilai yang berbeda. Oleh karena itu, kita akan isi nilai nan pada fashion_news_frequency dengan nilai Not Yet.

In [ ]:
def fill_fn_freq_nan(row):
    if row.fashion_news_frequency is not np.nan:
        return row.fashion_news_frequency

    if row.FN == 0:
        return "Never"

    if (row.FN == 1) & (row.club_member_status == "ACTIVE"):
        return "Other"
    if (row.FN == 1) & (row.club_member_status is np.nan):
        return "Other"

    if (row.FN == 1) & (row.club_member_status == "PRE-CREATE"):
        return "Not Yet"

In [ ]:
# Terapkan function tersebut pada kolom "fashion_news_frequency"
df_customers["fashion_news_frequency"] = df_customers.apply(fill_fn_freq_nan, axis=1)

In [ ]:
print(df_customers[["FN", "club_member_status", "fashion_news_frequency"]].value_counts())
print('------')
df_customers.info()

Berdasarkan info Non-Null di atas, fitur selanjutnya yang perlu dilihat missing values yaitu pada fitur club_member_status.

In [ ]:
# Buat function berdasarkan kondisi di atas
def fill_member_status_nan(row):
    if row.club_member_status is not np.nan:
        return row.club_member_status

    if row.Active == 0:
        return "NON-ACTIVE"
    return "UNKNOWN"

In [ ]:
# Terapkan function apply() pada kolom club_member_status
df_customers["club_member_status"] = df_customers.apply(fill_member_status_nan, axis=1)

In [ ]:
print(df_customers[["Active", "club_member_status"]].value_counts())
df_customers.info()

Selanjutnya, fitur yang masih memiliki missing values yaitu fitur age.
untuk penyederhanaan, kita akan isi dengan nilai median fitur tersebut.

In [ ]:
df_customers["age"] = df_customers.age.fillna(df_customers.age.median())

In [ ]:
df_customers.info()

### Data Visualization

In [ ]:
plot_count(x="FN", data=df_customers)
plt.xticks([0, 1], ["Not Subscribed", "Subscribed"])
plt.show()

In [ ]:
plot_count(x="Active", data=df_customers)
plt.xticks([0, 1], ["Not Active", "Active"])
plt.show()

In [ ]:
plot_count(x="club_member_status", data=df_customers)
plt.show()

In [ ]:
plot_count(x="fashion_news_frequency", data=df_customers)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x="age", data=df_customers)
plt.show()

Dari visualisasi boxplot di atas, terlihat bahwa terdapat pencilan di mana umur customer lebih dari 80 tahun. Ini mungkin sangat menarik untuk ditinjau lebih jauh lagi bagaimana transaksi yang mereka lakukan yang tercermin dalam data transaksi.
Kita bisa eksplor lebih jauh lagi, seperti membuat visualisasi yang mengombinasikan beberapa fitur kategorik pada data customer.

## EDA DATA TRANSACTIONS

Jika kita perhatikan sekali lagi informasi data transaksi di bawah terdapat lebih dari 31 juta baris transaksi dan untuk mengolah data sebesar itu, kita memerlukan resource yang lebih besar juga, seperti RAM, CPU, GPU, dan mungkin library yang lebih mutakhir. Oleh karena itu, pada meeting ini kita hanya akan menggunakan data transaksi di tahun 2020

* Kita akan lakukan join antara data artikel dan customer dengan data transkasi. Kita bisa menggunakan merge atau join dari Pandas.
* Tapi, sebelum itu, kita akan coba mengubah beberapa tipe data di data transaksi yang mungkin kurang tepat.

In [ ]:
df_transactions["t_dat"] = df_transactions.t_dat.astype("datetime64")
df_transactions["article_id"] = df_transactions.article_id.astype("object")
df_transactions.info()

1. Untuk melakukan join, data transaksi akan bertindak sebagai right table dan data artikel dan customer akan bertindak sebagai left table.
2. Selain itu, karena kita tidak ingin kehilangan data transaksi meskipun (mungkin) tidak memiliki pasangan di data artikel ataupun customer, kita akan menggunakan prinsip right join.
3. Karena alasan keterbatasan RAM Google Colab, kita akan menggunakan sebagian data transaksi saja, yaitu data transaksi dari bulan Juni 2020 sampai transaksi terakhir

In [ ]:
# Filter data transaksi hanya untuk juni 2020 karena datanya terlalu besar
df_transactions = df_transactions[df_transactions.t_dat >= "2020-06"].reset_index(drop=True)
df_transactions.info() 

In [ ]:
# Lakukan merge() pada artikel dengan transaksi
df_article_transactions = df_articles.merge(df_transactions, how="right", on="article_id")

# Lakukan merge() pada gabungan artikel transaksi dengan data customer
df_all = df_customers.merge(df_article_transactions, how="right", on="customer_id")

In [ ]:
df_all.info()

In [ ]:
df_all.isna().sum()

In [ ]:
# Tampilkan dari gabungan ke-3 data (artikel, customer, dan transaksi)
df_all.head()

In [ ]:
#Melihat unique values

num_customers = df_all.customer_id.nunique()
num_articles = df_all.article_id.nunique()

print("Number of unique customers:", num_customers)
print("Number of sold items:", num_articles)

Insight :

Dari informasi di atas, sebanyak ~5 juta pengguna yang bertransaksi mulai dari bulan Juni 2020 dengan total ~45 ribu produk terjual. Kemudian untuk eksplorasi selanjutnya akan mengelompokkan data berdasarkan 3 hal:
1. tanggal transaksi
2. artikel
3. customer

In [ ]:
df_by_day = df_all.groupby("t_dat")
df_by_month = df_all.groupby(df_all.t_dat.dt.month)
df_by_customers = df_all.groupby("customer_id")
df_by_articles = df_all.groupby("article_id")


In [ ]:
df_by_customers

##### Daily Sales

In [ ]:
daily_sales = df_by_day.agg(daily_sales=pd.NamedAgg(column="price", aggfunc="sum")).reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x="t_dat", y="daily_sales", data=daily_sales)
plt.xlabel("transaction date")
plt.ylabel("daily sales")
plt.show()

In [ ]:
monthly_sales = df_by_month.agg(monthly_sales=pd.NamedAgg(column="price", aggfunc="sum")).reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(x="t_dat", y="monthly_sales", data=monthly_sales)
plt.xticks([6, 7, 8, 9], ["June", "July", "August", "September"])
plt.ylabel("total sales")
plt.show()

Selanjutnya, untuk melihat behavior dari tiap customer dalam melakukan transaksi, kita akan coba untuk mendapatkan fitur seperti:
- jumlah total artikel yang dibeli
- jumlah jenis artikel yang dibeli (unique)
- total uang yang dikeluarkan
- rata-rata transaksi
- jumlah hari transaksi
- jumlah varian warna

In [ ]:
df_customers_agg = df_by_customers.agg(
    total_items_bought=pd.NamedAgg(
        column="article_id", aggfunc=lambda x: len(list(x))
    ),
    count_items_bought=pd.NamedAgg(
        column="article_id", aggfunc=lambda x: len(set(x))
    ),
    total_spending=pd.NamedAgg(
        column="price", aggfunc=lambda x: sum(x)
    ),
    avg_spending=pd.NamedAgg(
        column="price", aggfunc=lambda x: np.mean(x)
    ),
    count_buying_day=pd.NamedAgg(
        column="t_dat", aggfunc=lambda x: len(set(x))
    ),
    count_colour_bought=pd.NamedAgg(
        column="perceived_colour_master_name", aggfunc=lambda x: len(set(x))
    ),
).reset_index()

In [ ]:
# Summary statistics dari customer transaction
df_customers_agg.describe().T.round(3)

In [ ]:
# Visualisasikan hubungan antara variabel numerik pada df_customers_agg
sns.heatmap(df_customers_agg.corr(), annot=True)
plt.show()

Selanjutnya, dari data transaksi per artikel (produk), kita akan coba untuk mendapatkan fitur seperti:
- total penjualan
- gmv (Gross Merchandise Value) per produk
- jumlah customer yang membeli produk tersebut

In [ ]:
df_articles_agg = df_by_articles.agg(
    
    total_sold=pd.NamedAgg(column="article_id", aggfunc="count"),
    
    gmv_per_product=pd.NamedAgg(column="price", aggfunc=lambda x: sum(x)*len(x)),
    
    count_customer_bought=pd.NamedAgg(
        
        column="customer_id", aggfunc=lambda x: len(set(x))
    )
).reset_index()

In [ ]:
df_articles_agg.tail (5)

In [ ]:
df_articles_agg.describe().round(2)

In [ ]:
# Visualisasikan hubungan antara variabel numerik pada df_articles_agg
sns.heatmap(df_articles_agg.corr(), annot=True)
plt.show()

## MODEL DEVELOPMENT

In [ ]:
df_articles